# Utilisation de la bibliotheque mediapipe pour detecter les doigts
## Exemple de comptage des doigts

In [ ]:
%pip install mediapipe opencv-python pyautogui

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Step 3: Initialize MediaPipe Hands and Drawing Utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Step 4: Function to determine hand orientation
def determine_hand(hand_landmarks):
    # Check whether the pinky (20) is higher than the thumb (4) in the y-axis
    if hand_landmarks[20].x > hand_landmarks[4].x:
        return "main droite"  # Right hand
    else:
        return "main gauche"  # Left hand

# Function to count raised fingers based on landmarks
def count_fingers(hand_landmarks):
    fingers = []
    main = determine_hand(hand_landmarks)

    # Thumb
    if main == "main droite":
        if hand_landmarks[4].x < hand_landmarks[3].x:  # Thumb is raised if to the left
            fingers.append(1)
        else:
            fingers.append(0)
    else:  # Left hand
        if hand_landmarks[4].x > hand_landmarks[3].x:  # Thumb is raised if to the right
            fingers.append(1)
        else:
            fingers.append(0)

    # Other four fingers
    for tip in [8, 12, 16, 20]:  # Index, middle, ring, pinky
        if hand_landmarks[tip].y < hand_landmarks[tip - 2].y:  # Finger is raised
            fingers.append(1)
        else:
            fingers.append(0)

    return fingers.count(1)

# Step 5: Webcam setup and hand sign detection
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty frame.")
            continue

        # Flip the image horizontally for a later selfie-view display
        # Convert the BGR image to RGB
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image and detect hand landmarks
        results = hands.process(image)

        # Convert the image back to BGR for OpenCV processing
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the image
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Count fingers
                num_fingers = count_fingers(hand_landmarks.landmark)

                # Display the number of fingers
                cv2.putText(image, f'Fingers: {num_fingers}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (255, 0, 0), 2, cv2.LINE_AA)

        # Display the final output
        cv2.imshow('Hand Sign Detection', image)

        if cv2.waitKey(5) & 0xFF == 27:  # Exit when 'Escape' key is pressed
            break

cap.release()
cv2.destroyAllWindows()


# Example control mouse with finger

In [ ]:
%pip install mediapipe opencv-python pycaw

In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np

# Step 3: Initialize MediaPipe Hands and Drawing Utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Get screen size
screen_width, screen_height = pyautogui.size()

# Step 4: Webcam setup and finger-based mouse control
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty frame.")
            continue

        # Flip the image horizontally for a selfie-view display
        # Convert the BGR image to RGB
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        #image en mode read-only pour un processing rapide de chaque image
        image.flags.writeable = False

        # Process the image and detect hand landmarks
        results = hands.process(image)

        # Convert the image back to BGR for OpenCV processing
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # If hand landmarks are detected, move the mouse based on index finger position
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the image
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get the index finger tip coordinates (landmark 8)
                index_finger_tip = hand_landmarks.landmark[8]
                if hand_landmarks.landmark[8].y<hand_landmarks.landmark[5].y  or hand_landmarks.landmark[8].y<hand_landmarks.landmark[6].y:
                    h, w, c = image.shape
                    finger_x = int(index_finger_tip.x * w)
                    finger_y = int(index_finger_tip.y * h)
    
                    # Convert the finger coordinates from webcam resolution to screen resolution
                    screen_x = np.interp(finger_x, [0, w], [0, screen_width])
                    screen_y = np.interp(finger_y, [0, h], [0, screen_height])
    
                    # Move the mouse cursor to the finger position
                    pyautogui.moveTo(screen_x, screen_y)
    
                    # Draw a circle on the index finger tip
                    cv2.circle(image, (finger_x, finger_y), 10, (0, 0, 255), cv2.FILLED)

        # Display the webcam feed with hand tracking
        cv2.imshow('Finger Mouse Control', image)

        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


# Example volume control with fingers 

In [ ]:
%pip install mediapipe opencv-python pyautogui

In [ ]:
import cv2
import mediapipe as mp
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

# Step 3: Initialize MediaPipe Hands and pycaw for volume control
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize pycaw for volume control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Get volume range (min, max)
vol_range = volume.GetVolumeRange()
min_vol = vol_range[0]
max_vol = vol_range[1]

# Step 4: Function to calculate the distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

# Step 5: Webcam setup for hand detection
cap = cv2.VideoCapture(0)

# Deadzone for setting volume to 0
deadzone = 20  # Distance in pixels considered "touching" (you can adjust this)

# Initialize MediaPipe Hands
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty frame.")
            continue

        # Flip the image horizontally for a selfie-view display
        # Convert the BGR image to RGB
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image and detect hand landmarks
        results = hands.process(image)

        # Convert the image back to BGR for OpenCV processing
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # If hand landmarks are detected, process them
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the image
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get thumb tip and index finger tip coordinates
                thumb_tip = hand_landmarks.landmark[4]   # Thumb tip
                index_finger_tip = hand_landmarks.landmark[8]  # Index finger tip

                # Get height and width of the image
                h, w, c = image.shape

                # Convert normalized landmarks to pixel values
                thumb_x, thumb_y = int(thumb_tip.x * w), int(thumb_tip.y * h)
                index_x, index_y = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
                print('Pt 4',hand_landmarks.landmark[4].y)
                print('Pt 10',hand_landmarks.landmark[10].y)
                if hand_landmarks.landmark[8].y<hand_landmarks.landmark[5].y:
                    # Calculate the distance between thumb and index finger
                    distance = calculate_distance(thumb_x, thumb_y, index_x, index_y)

                    # Set volume to 0 if the distance is within the deadzone
                    if distance < deadzone:
                        vol = min_vol  # Set to minimum volume
                    else:
                        # Map the distance to volume (0 when touching, max when far apart)
                        max_distance = 200  # Adjust this based on your hand size and webcam resolution
                        vol = np.interp(distance, [deadzone, max_distance], [min_vol, max_vol])

                    # Set the system volume based on the calculated volume
                    volume.SetMasterVolumeLevel(vol, None)

                    # Draw circles on thumb and index finger tips
                    cv2.circle(image, (thumb_x, thumb_y), 10, (255, 0, 0), cv2.FILLED)
                    cv2.circle(image, (index_x, index_y), 10, (255, 0, 0), cv2.FILLED)

                    # Draw a line between thumb and index finger
                    cv2.line(image, (thumb_x, thumb_y), (index_x, index_y), (0, 255, 0), 3)

                    # Display the distance (for debugging purposes)
                    cv2.putText(image, f'Distance: {int(distance)}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                    # Display the volume level (for debugging purposes)
                    cv2.putText(image, f'Volume: {int(np.interp(vol, [min_vol, max_vol], [0, 100]))}%', 
                                (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Display the webcam feed
        cv2.imshow('Volume Control with Hand Gestures', image)

        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


# Volume avec angle

In [5]:
import cv2
import mediapipe as mp
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

# Step 1: Initialize MediaPipe Hands and pycaw for volume control
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize pycaw for volume control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Get volume range (min, max)
vol_range = volume.GetVolumeRange()
min_vol = vol_range[0]
max_vol = vol_range[1]

# Step 2: Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point (vertex)
    c = np.array(c)  # Third point

    # Calculate vectors
    ab = a - b
    bc = c - b

    # Dot product and magnitudes
    dot_product = np.dot(ab, bc)
    magnitude = np.linalg.norm(ab) * np.linalg.norm(bc)

    # Calculate angle in radians and convert to degrees
    angle = np.arccos(dot_product / magnitude)
    return np.degrees(angle)

# Step 3: Webcam setup for hand detection
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty frame.")
            continue

        # Flip the image horizontally for a selfie-view display
        # Convert the BGR image to RGB
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image and detect hand landmarks
        results = hands.process(image)

        # Convert the image back to BGR for OpenCV processing
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Step 4: Process detected landmarks
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the image
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get wrist, thumb tip, and index finger tip coordinates
                wrist = hand_landmarks.landmark[0]  # Wrist
                thumb_tip = hand_landmarks.landmark[4]  # Thumb tip
                index_tip = hand_landmarks.landmark[8]  # Index finger tip

                # Get height and width of the image
                h, w, c = image.shape

                # Convert normalized landmarks to pixel coordinates
                wrist_coords = [int(wrist.x * w), int(wrist.y * h)]
                thumb_coords = [int(thumb_tip.x * w), int(thumb_tip.y * h)]
                index_coords = [int(index_tip.x * w), int(index_tip.y * h)]

                if hand_landmarks.landmark[8].y<hand_landmarks.landmark[5].y:
                    
                    # Calculate the angle
                    angle = calculate_angle(thumb_coords, wrist_coords, index_coords)

                    # Map the angle to volume range
                    max_angle = 45  # Max angle for fully extended fingers
                    vol = np.interp(angle, [10, max_angle], [min_vol, max_vol])

                    # Set system volume
                    volume.SetMasterVolumeLevel(vol, None)

                    # Step 5: Visual feedback
                    # Display angle
                    cv2.putText(image, f'Angle: {int(angle)} deg', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                    # Display volume level
                    cv2.putText(image, f'Volume: {int(np.interp(vol, [min_vol, max_vol], [0, 100]))}%', 
                                (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

                    # Draw circles and lines on the image
                    cv2.circle(image, tuple(thumb_coords), 10, (255, 0, 0), cv2.FILLED)
                    cv2.circle(image, tuple(index_coords), 10, (255, 0, 0), cv2.FILLED)
                    cv2.circle(image, tuple(wrist_coords), 10, (0, 255, 0), cv2.FILLED)
                    cv2.line(image, tuple(wrist_coords), tuple(thumb_coords), (0, 255, 0), 3)
                    cv2.line(image, tuple(wrist_coords), tuple(index_coords), (0, 255, 0), 3)

        # Display the webcam feed
        cv2.imshow('Volume Control with Hand Gestures', image)

        # Press 'ESC' to exit
        if cv2.waitKey(5) & 0xFF == 27:
            break

# Release resources
cap.release()
cv2.destroyAllWindows()

# Example volume control with fingers v1.0 (sans distorion de prespective)

In [ ]:
import cv2
import mediapipe as mp
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

# Initialize MediaPipe Hands and Drawing Utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize pycaw for volume control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Get volume range (min, max)
min_vol, max_vol, _ = volume.GetVolumeRange()

# Function to calculate the distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

# Webcam setup
cap = cv2.VideoCapture(0)

# Deadzone and max distance for normalized mapping
DEADZONE = 0.1  # Normalized distance considered "touching"
MAX_NORMALIZED_DISTANCE = 0.3  # Adjust based on hand size and resolution

# Initialize MediaPipe Hands
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty frame.")
            continue

        # Flip and process the frame
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_frame.flags.writeable = False

        # Detect hand landmarks
        results = hands.process(rgb_frame)

        # Convert frame back to BGR for display
        rgb_frame.flags.writeable = True
        h, w, _ = frame.shape

        # Process detected hand landmarks
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks and connections
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get coordinates for thumb tip, index finger tip, and wrist
                thumb_tip = hand_landmarks.landmark[4]
                index_tip = hand_landmarks.landmark[8]
                wrist = hand_landmarks.landmark[0]

                # Calculate absolute distances (pixel values)
                thumb_x, thumb_y = int(thumb_tip.x * w), int(thumb_tip.y * h)
                index_x, index_y = int(index_tip.x * w), int(index_tip.y * h)
                wrist_x, wrist_y = int(wrist.x * w), int(wrist.y * h)

                # Calculate reference distance (wrist to middle finger tip)
                ref_distance = calculate_distance(wrist.x, wrist.y, hand_landmarks.landmark[12].x, hand_landmarks.landmark[12].y)

                # Calculate normalized distance between thumb and index finger
                normalized_distance = calculate_distance(thumb_tip.x, thumb_tip.y, index_tip.x, index_tip.y) / ref_distance

                # Set volume based on normalized distance
                if normalized_distance < DEADZONE:
                    vol = min_vol  # Minimum volume
                else:
                    vol = np.interp(normalized_distance, [DEADZONE, MAX_NORMALIZED_DISTANCE], [min_vol, max_vol])

                # Apply volume change
                volume.SetMasterVolumeLevel(vol, None)

                # Draw thumb, index markers, and line between them
                cv2.circle(frame, (thumb_x, thumb_y), 10, (255, 0, 0), cv2.FILLED)
                cv2.circle(frame, (index_x, index_y), 10, (255, 0, 0), cv2.FILLED)
                cv2.line(frame, (thumb_x, thumb_y), (index_x, index_y), (0, 255, 0), 3)

                # Display normalized distance and volume for debugging
                volume_percentage = int(np.interp(vol, [min_vol, max_vol], [0, 100]))
                cv2.putText(frame, f'Norm Dist: {normalized_distance:.2f}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (255, 255, 255), 2)
                cv2.putText(frame, f'Volume: {volume_percentage}%', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (255, 255, 255), 2)

        # Display the frame
        cv2.imshow('Volume Control with Hand Gestures', frame)

        # Exit on pressing 'Esc'
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


# Swipe up for open apps, down for desktop, left right for switching apps

In [7]:
import cv2
import mediapipe as mp
import time
import pyautogui
import numpy as np

# Step 3: Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Step 4: Webcam setup for hand detection
cap = cv2.VideoCapture(0)

# Gesture recognition variables
swipe_threshold = 100  # Distance in pixels to recognize a swipe
last_swipe_time = 0  # Time of last swipe
swipe_timeout = 1  # Time in seconds to limit swipes

# Initialize MediaPipe Hands
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    last_position = None  # To track the last hand position
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty frame.")
            continue

        # Flip the image horizontally for a selfie-view display
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image and detect hand landmarks
        results = hands.process(image)

        # Convert the image back to BGR for OpenCV processing
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # If hand landmarks are detected, process them
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the image
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get index finger tip coordinates
                index_finger_tip = hand_landmarks.landmark[8]  # Index finger tip

                # Get height and width of the image
                h, w, c = image.shape

                # Convert normalized landmark to pixel values
                index_x, index_y = int(index_finger_tip.x * w), int(index_finger_tip.y * h)

                # Calculate the swipe direction based on previous position
                if last_position is not None:
                    distance_y = last_position[1] - index_y  # Difference in Y position
                    distance_x = last_position[0] - index_x  # Difference in X position

                    # Check if the swipe distance is significant
                    if abs(distance_y) > swipe_threshold:
                        current_time = time.time()
                        if current_time - last_swipe_time > swipe_timeout:
                            last_swipe_time = current_time
                            if distance_y > swipe_threshold:  # Swipe Up
                                pyautogui.hotkey('win', 'tab')  # Show all apps
                            elif distance_y < -swipe_threshold:  # Swipe Down
                                pyautogui.hotkey('win', 'd')  # Show desktop

                    # Check left/right swipe
                    if abs(distance_x) > swipe_threshold:
                        current_time = time.time()
                        if current_time - last_swipe_time > swipe_timeout:
                            last_swipe_time = current_time
                            if distance_x > swipe_threshold:  # Swipe Right
                                pyautogui.hotkey('alt', 'tab')  # Switch to the next app
                            elif distance_x < -swipe_threshold:  # Swipe Left
                                pyautogui.hotkey('shift', 'alt', 'tab')  # Switch to the previous app

                # Update the last position
                last_position = (index_x, index_y)

        # Display the webcam feed
        cv2.imshow('Gesture Control for Apps', image)

        if cv2.waitKey(5) & 0xFF == 27:  # Press 'Esc' to exit
            break

cap.release()
cv2.destroyAllWindows()

# Pong game

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import random

# Initialize Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Pong game parameters
WIDTH, HEIGHT = 640, 480
PADDLE_WIDTH, PADDLE_HEIGHT = 20, 100
BALL_RADIUS = 10
PADDLE_SPEED = 20
ball_speed_x, ball_speed_y = 7, 7

# Initialize paddle and ball positions
player_y = HEIGHT // 2 - PADDLE_HEIGHT // 2
ball_x, ball_y = WIDTH // 2, HEIGHT // 2

# Create a function to detect finger position
def detect_finger_position(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)
    
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Get index finger tip position (landmark 8)
            x = int(hand_landmarks.landmark[8].x * WIDTH)
            y = int(hand_landmarks.landmark[8].y * HEIGHT)
            return x, y
    return None, None

# Game loop
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip frame for natural movement
    frame = cv2.flip(frame, 1)

    # Detect finger position
    finger_x, finger_y = detect_finger_position(frame)

    # Move the paddle
    if finger_y is not None:
        player_y = finger_y - PADDLE_HEIGHT // 2

    # Bound the paddle within the screen
    if player_y < 0:
        player_y = 0
    if player_y + PADDLE_HEIGHT > HEIGHT:
        player_y = HEIGHT - PADDLE_HEIGHT

    # Ball movement
    ball_x += ball_speed_x
    ball_y += ball_speed_y

    # Ball collision with top and bottom
    if ball_y - BALL_RADIUS <= 0 or ball_y + BALL_RADIUS >= HEIGHT:
        ball_speed_y *= -1

    # Ball collision with player paddle
    if (ball_x - BALL_RADIUS <= PADDLE_WIDTH and 
        player_y <= ball_y <= player_y + PADDLE_HEIGHT):
        ball_speed_x *= -1

    # Ball collision with right wall (rebound)
    if ball_x + BALL_RADIUS >= WIDTH:
        ball_speed_x *= -1

    # Drawing the paddle in red
    cv2.rectangle(frame, (0, player_y), (PADDLE_WIDTH, player_y + PADDLE_HEIGHT), (0, 0, 255), -1)
    
    # Drawing the ball in red
    cv2.circle(frame, (ball_x, ball_y), BALL_RADIUS, (0, 0, 255), -1)

    # Display the frame
    cv2.imshow('Pong Game', frame)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()